<a href="https://colab.research.google.com/github/tiagosilveiraa/portifoliods/blob/main/Recomenda%C3%A7%C3%A3o_de_Jogadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import requests
import asyncio
from datetime import date
import ipywidgets as widgets
from IPython.display import display

In [2]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

#1º Parametrização



In [3]:
leagues = requests.get('https://api.sofascore.com/api/v1/config/unique-tournaments/BR/football').json()['uniqueTournaments']
seasons = []

In [4]:
dp_leagues_data = []
dp_seasons_data = []
for l in leagues:  
  dp_leagues_data.append((l['name'], l['id']))
  s = requests.get(f'https://api.sofascore.com/api/v1/unique-tournament/{l["id"]}/seasons').json()['seasons']  
  for si in s:
    si['league'] = l['id']
    dp_seasons_data.append((si['year'], si['id'], si['league']))      

No menu abaixo selecione as ligas e temporadas que deseja obter os dados.

In [5]:
add_button = widgets.Button(description="Adicionar")
clear_button = widgets.Button(description="Limpar Seleções")
dp_leagues = widgets.Dropdown(options=dp_leagues_data, value=None)
dp_seasons = widgets.Dropdown(value=None)
output = widgets.Output()
selected_leagues = []
selected_seasons = []

def on_change(event):
  dp_seasons_league = []
  if event['name'] == 'value':
    for ds in dp_seasons_data:
      #Busco apenas pelas seasons da liga selecionada pela posição na tupla
      if ds[2] == event['new']: 
        dp_seasons_league.append(ds[0:2])
    dp_seasons.options = dp_seasons_league

def add_league(b):
  if dp_leagues.value == None or dp_seasons.value == None:
    output.clear_output()
    print('Informações não preenchidas!')
  elif dp_leagues.value in selected_leagues and dp_seasons.value in selected_seasons:
    with output:      
      output.clear_output()
      print(f"Erro: Liga e temporada já adicionadas")       
  else:
    selected_leagues.append(dp_leagues.value)
    selected_seasons.append(dp_seasons.value) 
    output.clear_output()   
    with output:      
      print('Liga adicionada com sucesso! :D')

def clear_selection(b):
  selected_leagues.clear()
  selected_seasons.clear()
  with output:      
      print('Parâmetros reiniciados.')
      
dp_leagues.observe(on_change)
add_button.on_click(add_league)
clear_button.on_click(clear_selection)
display(dp_leagues, dp_seasons, add_button, clear_button,   output)

Dropdown(options=(('European Championship', 1), ('Copa América', 133), ('World Cup', 16), ('UEFA Champions Lea…

Dropdown(options=(), value=None)

Button(description='Adicionar', style=ButtonStyle())

Button(description='Limpar Seleções', style=ButtonStyle())

Output()

Agora selecione as posições dos jogadores que terão seus dados extraídos.

*Para selecionar mais de uma posição use as teclas CRTL ou SHIFT*

In [6]:
dp_positions = widgets.SelectMultiple(
    options=[('Goleiros', 'G'), ('Defensores', 'D'), ('Meio-Campistas', 'M'), ('Atacantes','F')],    
    value=['G', 'D', 'M', 'F'],
    #rows=10,
    #description='Posições',
    disabled=False
)
display(dp_positions)

SelectMultiple(index=(0, 1, 2, 3), options=(('Goleiros', 'G'), ('Defensores', 'D'), ('Meio-Campistas', 'M'), (…

#2º Obtenção das estatisticas
Nesta etapa iremos utilizar os parâmetros definidos acima para capturar os dados no SofaScore

In [7]:
def make_url_stats(competition, season, position, offset):
  return f"https://api.sofascore.com/api/v1/unique-tournament/{competition}/season/{season}/statistics?offset={offset}&order=-rating&accumulation=total&filters=position.in.{position}&fields=%2CbigChancesMissed%2CsuccessfulDribbles%2CsuccessfulDribblesPercentage%2CtotalShots%2CshotsOnTarget%2CshotsOffTarget%2CblockedShots%2CgoalConversionPercentage%2CpenaltiesTaken%2CpenaltyGoals%2CpenaltyWon%2CshotFromSetPiece%2CfreeKickGoal%2CgoalsFromInsideTheBox%2CgoalsFromOutsideTheBox%2CheadedGoals%2CleftFootGoals%2CrightFootGoal%2CshitWoodwork%2Coffsides%2CpenaltyConversion%2CsetPieceConversion%2Crating%2Cinterceptions%2CpenaltyConceded%2Cclearances%2CerrorLeadToGoal%2CerrorLeadToShot%2CownGoals%2CdribbledPast%2CcleanSheet%2C rating %2CbigChancesCreated%2Cassists %2CaccuratePasses%2CinaccuratePasses%2CtotalPasses%2CaccuratePassesPercentage %2CaccurateOwnHalfPasses%2CaccurateOppositionHalfPasses%2CaccurateFinalThirdPasses%2CkeyPasses%2CaccurateCrosses%2CaccurateCrossesPercentage%2CaccurateLongBalls%2CaccurateLongBallsPercentage%2CpassToAssist%2Crating %2CcleanSheet%2CpenaltyFaced%2CpenaltySave%2CsavedShotsFromInsideTheBox%2CsavedShotsFromOutsideTheBox%2CgoalsConcededInsideTheBox%2CgoalsConcededOutsideTheBox%2Cpunches%2CsuccessfulRunsOut%2ChighClaims%2CcrossesNotClaimed%2Crating %2CyellowCards%2CredCards%2CgroundDuelsWon%2CgroundDuelsWonPercentage%2CaerialDuelsWon%2CaerialDuelsWonPercentage%2CtotalDuelsWon%2CtotalDuelsWonPercentage%2CminutesPlayed%2CwasFouled%2Cfouls%2Cdispossessed%2CpossessionLost%2Cappearances%2CmatchesStarted%2Crating &limit=20"

In [8]:
def get_stats(leagues, seasons, positions):
  stats = pd.DataFrame()  
  for idx, l in enumerate(leagues):    
    for p in positions:
     iw = 0
     while True:  
       response = requests.get(make_url_stats(l, seasons[idx], p, iw))
       #Já percoreu todos os players então paro
       if response.json()['results'] == []:        
         break;

       response = response.json()['results']
       response = pd.json_normalize(response, max_level=6)
       response['position'] = p        
       stats = pd.concat([stats, response])    
       iw = iw + 20 
  return stats

In [9]:
players = get_stats(selected_leagues, selected_seasons, dp_positions.value)

In [10]:
players

,bigChancesMissed,successfulDribbles,successfulDribblesPercentage,totalShots,shotsOnTarget,shotsOffTarget,blockedShots,goalConversionPercentage,penaltiesTaken,penaltyGoals,penaltyWon,shotFromSetPiece,freeKickGoal,goalsFromInsideTheBox,goalsFromOutsideTheBox,headedGoals,leftFootGoals,offsides,penaltyConversion,setPieceConversion,rating,interceptions,penaltyConceded,clearances,errorLeadToGoal,errorLeadToShot,ownGoals,dribbledPast,cleanSheet,bigChancesCreated,accuratePasses,inaccuratePasses,totalPasses,accurateOwnHalfPasses,accurateOppositionHalfPasses,accurateFinalThirdPasses,keyPasses,accurateCrosses,accurateCrossesPercentage,accurateLongBalls,accurateLongBallsPercentage,passToAssist,penaltyFaced,penaltySave,savedShotsFromInsideTheBox,savedShotsFromOutsideTheBox,goalsConcededInsideTheBox,goalsConcededOutsideTheBox,punches,successfulRunsOut,highClaims,crossesNotClaimed,yellowCards,redCards,groundDuelsWon,groundDuelsWonPercentage,aerialDuelsWon,aerialDuelsWonPercentage,totalDuelsWon,totalDuelsWonPercentage,minutesPlayed,wasFouled,fouls,dispossessed,possessionLost,appearances,matchesStarted,player.name,player.slug,player.userCount,player.id,team.name,team.slug,team.shortName,team.sport.name,team.sport.slug,team.sport.id,team.userCount,team.type,team.id,team.teamColors.primary,team.teamColors.secondary,team.teamColors.text,position
0,0,0,0.00,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.60,0,0,3,0,0,0,0,1,0,11,11,22,10,1,0,0,0,0.0,2,15.38,0,0,0,2,1,0,0,2,0,2,0,0,0,0,0.00,2,66.67,2,66.67,90,0,0,0,11,1,1,Jandrei,jandrei,343,874979,Santos,santos,Santos,Football,football,1,0,0,1968,#52b030,#52b030,#ffffff,G
1,0,0,0.00,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.53,0,0,5,0,0,0,1,2,0,54,62,116,29,25,3,0,0,0.0,30,32.97,0,0,0,13,6,3,1,3,1,1,1,0,0,0,0.00,1,50.00,1,33.33,360,0,0,0,63,4,4,Vinicius Silvestre,vinicius-silvestre,232,559582,Palmeiras,palmeiras,Palmeiras,Football,football,1,0,0,1963,#52b030,#52b030,#ffffff,G
2,0,0,0.00,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.40,0,0,0,0,0,0,0,1,0,6,0,6,6,0,0,0,0,0.0,1,100.00,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0.00,0,0.00,0,0.00,90,0,0,0,0,1,1,Matheus Donelli,matheus-donelli,402,1002609,Corinthians,corinthians,Corinthians,Football,football,1,0,0,1957,#52b030,#52b030,#ffffff,G
3,0,0,0.00,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.32,0,0,10,0,0,0,0,6,0,221,101,322,158,63,16,0,0,0.0,108,51.92,0,0,0,29,14,7,2,6,2,4,0,4,0,1,100.00,3,100.00,4,100.00,1125,2,0,0,102,13,13,João Ricardo,joao-ricardo,187,839761,Ceará,ceara,Ceará,Football,football,1,0,0,2001,#52b030,#52b030,#ffffff,G
4,0,1,100.00,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.20,4,0,30,1,3,0,1,14,0,620,272,892,499,121,27,0,0,0.0,207,43.58,0,5,0,70,71,33,3,18,6,50,0,2,0,4,80.00,7,87.50,11,84.62,3330,3,0,0,286,37,37,Mailson,mailson,351,883367,Sport Recife,sport-recife,Sport Recife,Football,football,1,0,0,1959,#52b030,#52b030,#ffffff,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,0,0,0.00,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.25,1,0,0,0,0,0,1,0,0,11,6,17,5,6,3,0,0,0.0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,25.00,1,20.00,2,22.22,27,0,0,1,10,2,0,Matías Lacava,matias-lacava,269,1016029,Santos,santos,Santos,Football,football,1,0,0,1968,#52b030,#52b030,#ffffff,F
15,0,1,25.00,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.25,0,0,0,0,0,0,0,0,0,8,0,8,0,8,4,0,0,0.0,0,0.00,0,0,0,0,0,1,0,0,0,0,0,0,0,2,28.57,0,0.00,2,28.57,40,1,1,1,4,3,0,Gustavo Mantuan,gustavo-mantuan,654,1066790,Corinthians,corinthians,Corinthians,Football,football,1,0,0,1957,#52b030,#52b030,#ffffff,F
16,1,2,66.67,4,2,1,1,0.0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,6.25,0,0,1,0,0,0,1,0,0,34,6,40,11,23,14,1,0,0.0,1,50.00,0,0,0,0,0,1,0,0,0,0,0,0,1,8,53.33,0,0.00,8,47.06,116,4,4,1,15,7,0,Maxwell,maxwell,43,880134,Sport Recife,sport-r

#3º - Enriquecimento dos dados
Na etapa anterior obtivemos todas as estatisticas disponíveis para os jogadores, agora vamos fazer o enriquecimento desses dados com outras informações.

Buscando por posições alternativas para cada player, bem como algumas caracteristicas, como altura, valor de mercado e etc...

In [11]:
async def parallel_requests(coroutines):
  await asyncio.gather(*coroutines)

In [12]:
players_atributtes = []
players_alternative_positions = []

async def get_player_attributes(player_id):
  response_data = requests.get(f'https://api.sofascore.com/api/v1/player/{player_id}')
  response = response_data.json()['player']
  response['player.id'] = player_id #Para fazer o merge depois
  players_atributtes.append(response)

async def get_alternative_positions(player_id):
  response_data = requests.get(f'https://api.sofascore.com/api/v1/player/{player_id}/characteristics')
  response = response_data.json()
  if len(response['positions']) >=1:
    response = {key: response[key] for key in response.keys() if key == 'positions'}
    response['player.id'] = player_id #Para fazer o merge depois
    players_alternative_positions.append(response)

In [13]:
req_attributes = [get_player_attributes(p) for p in players['player.id']]
req_alternative_positions = [get_alternative_positions(p) for p in players['player.id']]

In [14]:
await parallel_requests(req_alternative_positions)
await parallel_requests(req_attributes)

#2º Tratamento dos dados


In [15]:
df_players_attributes = pd.json_normalize(players_atributtes, max_level=2)
df_players_alternative_positions = pd.json_normalize(players_alternative_positions, max_level=2)

In [49]:
merged_data = df_players_alternative_positions.merge(df_players_attributes, on='player.id')

In [52]:
#Como vieram muitas informações redundantes iremos dropá-las
merged_data = merged_data.drop(columns=['position','name', 'firstName', 'lastName', 'slug', 'shortName','proposedMarketValue', 'jerseyNumber','team.slug', 'team.shortName', 'team.gender', 'team.sport.name',
       'team.sport.slug', 'team.sport.id', 'team.tournament.name',
       'team.tournament.slug', 'team.tournament.category',
       'team.tournament.uniqueTournament', 'team.tournament.priority',
       'team.tournament.id', 'team.primaryUniqueTournament.name',
       'team.primaryUniqueTournament.slug',
       'team.primaryUniqueTournament.category',
       'team.primaryUniqueTournament.userCount',
       'team.primaryUniqueTournament.id',
       'team.primaryUniqueTournament.displayInverseHomeAwayTeams',
       'team.userCount', 'team.nameCode', 'team.disabled', 'team.national',
       'team.type', 'team.id', 'team.country.alpha2', 'team.country.name',
       'team.teamColors.primary', 'team.teamColors.secondary',
       'team.teamColors.text', 'userCount', 'id', 'team.name'])

In [54]:
merged_data.rename(columns={'country.alpha2':'countryCode', 'country.name':'countryName', 'proposedMarketValueRaw.value':'proposedMarketValue', 'proposedMarketValueRaw.currency': 'proposedMarketCurrency'}, inplace=True)

In [55]:
df_players = merged_data.merge(players, on='player.id')

In [56]:
#colocando todas as colunas em minusculo
df_players = df_players.rename(str.lower,axis='columns')    

In [57]:
#Só quero players que estiveram em alguma partida na temporada
df_players = df_players[df_players['appearances'] >0]

In [58]:
#Alterando o indice para o player.id
df_players.set_index('player.id', inplace = True)

#2.1 - Tratando valores nulos

In [ ]:
df_clean.columns[df_clean.isna().any()].tolist()

In [70]:
df_players[df_players['height'].isna()]

,positions,height,preferredfoot,shirtnumber,dateofbirthtimestamp,contractuntiltimestamp,countrycode,countryname,proposedmarketvalue,proposedmarketcurrency,retired,bigchancesmissed,successfuldribbles,successfuldribblespercentage,totalshots,shotsontarget,shotsofftarget,blockedshots,goalconversionpercentage,penaltiestaken,penaltygoals,penaltywon,shotfromsetpiece,freekickgoal,goalsfrominsidethebox,goalsfromoutsidethebox,headedgoals,leftfootgoals,offsides,penaltyconversion,setpiececonversion,rating,interceptions,penaltyconceded,clearances,errorleadtogoal,errorleadtoshot,owngoals,dribbledpast,cleansheet,bigchancescreated,accuratepasses,inaccuratepasses,totalpasses,accurateownhalfpasses,accurateoppositionhalfpasses,accuratefinalthirdpasses,keypasses,accuratecrosses,accuratecrossespercentage,accuratelongballs,accuratelongballspercentage,passtoassist,penaltyfaced,penaltysave,savedshotsfrominsidethebox,savedshotsfromoutsidethebox,goalsconcededinsidethebox,goalsconcededoutsidethebox,punches,successfulrunsout,highclaims,crossesnotclaimed,yellowcards,redcards,groundduelswon,groundduelswonpercentage,aerialduelswon,aerialduelswonpercentage,totalduelswon,totalduelswonpercentage,minutesplayed,wasfouled,fouls,dispossessed,possessionlost,appearances,matchesstarted,player.name,player.slug,player.usercount,team.name,team.slug,team.shortname,team.sport.name,team.sport.slug,team.sport.id,team.usercount,team.type,team.id,team.teamcolors.primary,team.teamcolors.secondary,team.teamcolors.text,position
player.id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1020595,[ST],NaN,NaN,28.0,1.036541e+09,1.735603e+09,BR,Brazil,NaN,NaN,False,1,3,100.0,3,1,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.5,0,0,0,0,0,0,2,0,0,14,4,18,5,9,5,0,0,0.0,0,0.0,0,0,0,0,0,2,0,0,0,0,0,0,0,6,66.67,1,50.0,7,63.64,109,3,0,1,6,5,0,Felipe Felicio,felipe-felicio,73,Atlético Mineiro,atletico-mineiro,Atlético Mineiro,Football,football,1,0,0,1977,#52b030,#52b030,#ffffff,F


#3º Análise exploratória



#4° Geração do Modelo

In [ ]:
numeric_stats = players.drop([
    'team.name',
'player.name',
'player.slug',
'player.usercount',
'team.name',
'team.slug',
'team.shortname',
'team.sport.name',
'team.sport.slug',
'team.sport.id',
'team.usercount',
'team.type',
'team.id',
'team.teamcolors.primary',
'team.teamcolors.secondary',
'team.teamcolors.text',
'position',
'rating'
], axis='columns')

In [ ]:
numeric_stats.columns

In [ ]:
from scipy.sparse import csr_matrix
stats_sparse = csr_matrix(numeric_stats)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute', n_neighbors=5)
model.fit(stats_sparse)

In [ ]:
distance, similar_players_index = model.kneighbors(numeric_stats.loc[840219].values.reshape(1,-1))
similar_players = pd.DataFrame()
similar_players = players.loc[players.index[similar_players_index[0]]]
similar_players['distances'] = distance[0]

In [ ]:
similar_players